# Proyecto 2 Reconocimiento de Patrones

José Julián Camacho Hernández

Leonardo Guillén Fernández

In [90]:
import os
import cv2
import time
import math
import torch
import numpy as np
import pandas as pd
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from IPython.display import display
from torch.utils.data import DataLoader, Dataset
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, f1_score

# MLP

In [125]:
def computeMetrics(y_test, y_pred, training_time):
    acc = accuracy_score(y_test, y_pred)                        # Calcular la exactitud
    f1 = f1_score(y_test, y_pred, average='macro')              # Calcular F1 score
    rec = recall_score(y_test, y_pred, average='macro')         # Calcular el recall
    prec = precision_score(y_test, y_pred, average='macro')     # Calcular la precisión
    #auc = roc_auc_score(y_test, y_pred, average='macro')        # Calcular AUC , "AUC":auc}
    
    true_labels_binary = label_binarize(y_test, classes=[0,1,2,3])
    auc = roc_auc_score(true_labels_binary, y_pred.reshape(-1, 1), average='macro')

    metrics = {"Accuracy":acc, "Precision":prec, "Recall":rec, "F1 Score":f1, "AUC":auc, "Tiempo de entrenamiento":training_time}
    df = pd.DataFrame(metrics, index = [0])
    display(df)

    ## Calcular la curva ROC
    #fpr = dict()
    #tpr = dict()
    #roc_auc = dict()
    #for i in range(4):
    #    fpr[i], tpr[i], _ = roc_curve(true_labels_binary[:,i], y_pred[:,i])
    #    roc_auc[i] = auc(fpr[i], tpr[i])

    #plt.figure()

    #for i in range(4):
    #    plt.plot(fpr[i], tpr[i], label='Class {}: AUC = {:.2f}'.format(i, roc_auc[i]))

    #plt.plot([0, 1], [0, 1], 'k--')  # Plot the diagonal line
    #plt.xlim([0.0, 1.0])
    #plt.ylim([0.0, 1.05])
    #plt.xlabel('False Positive Rate')
    #plt.ylabel('True Positive Rate')
    #plt.title('Receiver Operating Characteristic')
    #plt.legend(loc='lower right')
    #plt.show()


## MLP sin feature extractor

In [55]:
#=====  Cargar y aplicar feature extractor a las imágenes  =====#
X = []
y = []

input_folders = ["./testi3/", "./testi4/", "./testi2/", "./testi/"]
#input_folders = ["./COVID-19_Radiography_Dataset/COVID/", "./COVID-19_Radiography_Dataset/Lung_Opacity/",
#                 "./COVID-19_Radiography_Dataset/Normal/", "./COVID-19_Radiography_Dataset/Viral_Pneumonia/"]
i = 0
for folder in input_folders:
    input_folder = folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = Image.open(image_path)
            image = image.convert('L')          # Convertir a escala de grises
            image = np.array(image).flatten()   # Flatten the image
            X.append(image)
            y.append(i)
    i+=1

np.save("X.npy", X)
np.save("y.npy", y)

In [129]:
#=====  Feature Engineering  =====#

try:
    X_charged = np.load("X.npy")
    y_charged = np.load("y.npy")
except FileNotFoundError:
    print("Error: File not found.")
       
# Convertir las listas a numpy array de tipo uint8
X_np = np.array(X_charged, dtype=np.uint8)
y_np = np.array(y_charged)

# Dividir los datos en train y test y estratificar
train_X, test_X, train_y, test_y = train_test_split(
    X_np, y_np, test_size=0.2, stratify=y_np, random_state=42)

# Normalización de los datos
scaler = MinMaxScaler()
scaler.fit(train_X)
scaled_data = scaler.transform(train_X)
train_X = scaled_data

# Convertir de numpy array a PyTorch tensor
X_train_tensor = torch.from_numpy(train_X).float()
X_test_tensor = torch.from_numpy(test_X).float()
y_train_tensor = torch.from_numpy(train_y).float()
y_test_tensor = torch.from_numpy(test_y).float()

# Convertir a tensores de tipo long
y_train_long_tensor = y_train_tensor.long()         
y_train_long_tensor = y_train_long_tensor.to(device)

# Convertir a one hot
y_onehot = F.one_hot(y_train_long_tensor, num_classes=num_classes).float()


In [58]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, activation):
        super(MLP, self).__init__()
        self.input_size = input_size
        self.hidden_layers = nn.ModuleList()
        self.num_classes = num_classes
        self.activation = activation
        # Agregar capas ocultas
        for i, hidden_size in enumerate(hidden_sizes):
            if i == 0:
                self.hidden_layers.append(nn.Linear(input_size, hidden_size))
            else:
                self.hidden_layers.append(nn.Linear(hidden_sizes[i-1], hidden_size))
        # Capa de salida
        self.output_layer = nn.Linear(hidden_sizes[-1], num_classes)
        
    def forward(self, x):
        out = x
        # Pasar por capas ocultas
        for layer in self.hidden_layers:
            out = layer(out)
            out = self.activation(out)
        # Pasar por capa de salida
        out = self.output_layer(out)
        return out


### Diseño # 1
Capas: [81401, 128, 64, 4] | Función de activación: ReLU | α = 0.001 | epochs = 1000

In [127]:
#=====  Instaciar modelo  =====#

input_size = X_train_tensor.shape[1]
hidden_sizes = [128, 64]       
num_classes = len(set(y_train_tensor))   
activation = F.relu
model = MLP(input_size, hidden_sizes, num_classes, activation)   

# Definir la función de loss con learning rate y optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start_time = time.time()
# Entrenar el modelo
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
end_time = time.time()
time_taken = end_time - start_time

# Evaluate the model
outputs = model(X_test_tensor)
_, predicted = torch.max(outputs.data, 1)

# Obtener métricas
computeMetrics(y_test_tensor, predicted, time_taken)
print('Resultados esperados: ', y_test_tensor)
print('Resultados obtenidos: ', predicted)


,Accuracy,Precision,Recall,F1 Score,AUC,Tiempo de entrenamiento
0,0.703704,0.746429,0.68125,0.686111,0.108553,124.830823


Resultados esperados:  tensor([1., 0., 0., 3., 2., 2., 0., 2., 0., 1., 3., 0., 2., 3., 1., 1., 3., 2.,
        2., 3., 0., 0., 1., 3., 0., 2., 2.])
Resultados obtenidos:  tensor([0, 1, 0, 3, 2, 2, 0, 2, 0, 1, 2, 0, 0, 1, 1, 1, 3, 2, 1, 3, 0, 0, 0, 1,
        0, 2, 2])


### Diseño # 2
Capas: [81401, 256, 128, 64, 4] | Función de activación: ReLU | α = 0.001 | epochs = 1000

In [82]:
#=====  Instaciar modelo  =====#

input_size = X_train_tensor.shape[1]
hidden_sizes = [256, 128, 64]       
num_classes = len(set(y_train_tensor))   
activation = F.relu
model = MLP(input_size, hidden_sizes, num_classes, activation)   

# Definir la función de loss con learning rate y optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start_time = time.time()
# Entrenar el modelo
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
end_time = time.time()
time_taken = end_time - start_time

# Evaluate the model
outputs = model(X_test_tensor)
_, predicted = torch.max(outputs.data, 1)

# Obtener métricas
computeMetrics(predicted, y_test_tensor, time_taken)
print('Resultados esperados: ', y_test_tensor)
print('Resultados obtenidos: ', predicted)


,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento
0,0.703704,0.691667,0.754167,0.703571,302.785814


Resultados esperados:  tensor([1., 0., 0., 3., 2., 2., 0., 2., 0., 1., 3., 0., 2., 3., 1., 1., 3., 2.,
        2., 3., 0., 0., 1., 3., 0., 2., 2.])
Resultados obtenidos:  tensor([0, 1, 0, 3, 2, 2, 0, 2, 0, 1, 2, 0, 0, 0, 1, 1, 3, 0, 1, 3, 0, 0, 0, 3,
        0, 2, 2])


### Diseño # 3
Capas: [81401, 128, 64, 4] | Función de activación: ReLU | α = 0.001 | epochs = 1000

In [130]:
#=====  Instaciar modelo  =====#

input_size = X_train_tensor.shape[1]
hidden_sizes = [150, 50]       
num_classes = len(set(y_train_tensor))   
activation = F.relu
model = MLP(input_size, hidden_sizes, num_classes, activation)   

# Definir la función de loss con learning rate y optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start_time = time.time()
# Entrenar el modelo
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
end_time = time.time()
time_taken = end_time - start_time

# Evaluate the model
outputs = model(X_test_tensor)
_, predicted = torch.max(outputs.data, 1)

# Obtener métricas
computeMetrics(y_test_tensor, predicted, time_taken)
print('Resultados esperados: ', y_test_tensor)
print('Resultados obtenidos: ', predicted)


,Accuracy,Precision,Recall,F1 Score,AUC,Tiempo de entrenamiento
0,0.703704,0.754167,0.691667,0.703571,0.161184,151.206025


Resultados esperados:  tensor([1., 0., 0., 3., 2., 2., 0., 2., 0., 1., 3., 0., 2., 3., 1., 1., 3., 2.,
        2., 3., 0., 0., 1., 3., 0., 2., 2.])
Resultados obtenidos:  tensor([0, 1, 0, 3, 2, 2, 0, 2, 0, 1, 2, 0, 0, 0, 1, 1, 3, 0, 1, 3, 0, 0, 0, 3,
        0, 2, 2])


In [124]:
#=====  Instaciar modelo  =====#

input_size = X_train_tensor.shape[1]
hidden_sizes = [200, 50]       
num_classes = len(set(y_train_tensor))   
activation = F.relu
model = MLP(input_size, hidden_sizes, num_classes, activation)   

# Definir la función de loss con learning rate y optimizer
learning_rate = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

start_time = time.time()
# Entrenar el modelo
for epoch in range(20):
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_onehot)
    loss.backward()
    optimizer.step()
end_time = time.time()
time_taken = end_time - start_time

# Evaluate the model
outputs = model(X_test_tensor)
_, predicted = torch.max(outputs.data, 1)

# Obtener métricas
computeMetrics(y_test_tensor, predicted, time_taken)
print('Resultados esperados: ', y_test_tensor)
print('Resultados obtenidos: ', predicted)


,Accuracy,Precision,Recall,F1 Score,AUC,Tiempo de entrenamiento
0,0.62963,0.695513,0.63125,0.557143,0.131579,4.094419


IndexError: too many indices for tensor of dimension 1

## MLP con feature extractor LBP

In [7]:
#=====  Feature Extractor  =====#
def lbp(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp_image = np.zeros_like(gray)
    for i in range(1, gray.shape[0] - 1):
        for j in range(1, gray.shape[1] - 1):
            binary = ""
            center = gray[i, j]
            for x, y in [(-1, -1), (-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1)]:
                if gray[i + x, j + y] >= center:
                    binary += "1"
                else:
                    binary += "0"
            lbp_image[i, j] = int(binary, 2)
    return lbp_image.flatten()

In [16]:
#=====  Cargar y aplicar feature extractor a las imágenes  =====#
lbp_arrays = []
lbp_labels = []

input_folders = ["./testi3/", "./testi4/", "./testi2/", "./testi/"]
#input_folders = ["./COVID-19_Radiography_Dataset/COVID/", "./COVID-19_Radiography_Dataset/Lung_Opacity/",
#                 "./COVID-19_Radiography_Dataset/Normal/", "./COVID-19_Radiography_Dataset/Viral_Pneumonia/"]
i = 0
for folder in input_folders:
    input_folder = folder
    for filename in os.listdir(input_folder):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_path = os.path.join(input_folder, filename)
            image = cv2.imread(image_path)
            lbp_array = lbp(image)
            lbp_arrays.append(lbp_array)
            lbp_labels.append(i)
    i+=1

np.save("lbp_arrays.npy", lbp_arrays)
np.save("lbp_labels.npy", lbp_labels)

In [26]:
#=====  Feature Engineering  =====#

try:
    lbp_arrays_charged = np.load("lbp_arrays.npy")
    labels_charged = np.load("lbp_labels.npy")
except FileNotFoundError:
    print("Error: File not found.")
    
print(lbp_arrays_charged)    
print(labels_charged)    
# Convertir las listas a numpy array de tipo uint8
lbp_np_arrays = np.array(lbp_arrays_charged, dtype=np.uint8)
labels_np = np.array(labels_charged)

# Dividir los datos en train y test y estratificar
train_lbp, test_lbp, train_labels, test_labels = train_test_split(
    lbp_np_arrays, labels_np, test_size=0.2, stratify=labels_np, random_state=42)

# Normalización de los datos
scaler = MinMaxScaler()
scaler.fit(train_lbp)
scaled_data = scaler.transform(train_lbp)
train_lbp = scaled_data

# Convertir de numpy array a PyTorch tensor
lbp_train_tensor = torch.from_numpy(train_lbp).float()
lbp_test_tensor = torch.from_numpy(test_lbp).float()
labels_train_tensor = torch.from_numpy(train_labels).float()
labels_test_tensor = torch.from_numpy(test_labels).float()


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3]


In [29]:
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [30]:
# Instaciar modelo
input_size = lbp_train_tensor.shape[1]
hidden_size = 64
output_size = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MLP(input_size, hidden_size, output_size)

# Definir la función de loss y optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

start_time = time.time()
# Entrenar el modelo
for epoch in range(1000):
    optimizer.zero_grad()
    outputs = model(lbp_train_tensor)

    labels_train_long_tensor = labels_train_tensor.long()         
    labels_train_long_tensor = labels_train_long_tensor.to(device)
    labels_onehot = F.one_hot(labels_train_long_tensor, num_classes=output_size).float()

    loss = criterion(outputs, labels_onehot)
    loss.backward()
    optimizer.step()
end_time = time.time()
time_taken = end_time - start_time

# Evaluate the model
outputs = model(lbp_test_tensor)
_, predicted = torch.max(outputs.data, 1)

# Obtener métricas
computeMetrics(y_test_tensor, predicted, time_taken)
print('Resultados esperados: ', labels_test_tensor)
print('Resultados obtenidos: ', predicted)

,Accuracy,Precision,Recall,F1 Score,Tiempo de entrenamiento
0,0.777778,0.75625,0.783333,0.73953,67.009742


Resultados esperados:  tensor([1., 0., 0., 3., 2., 2., 0., 2., 0., 1., 3., 0., 2., 3., 1., 1., 3., 2.,
        2., 3., 0., 0., 1., 3., 0., 2., 2.])
Resultados obtenidos:  tensor([0, 0, 0, 3, 2, 2, 0, 2, 0, 1, 3, 0, 3, 3, 1, 3, 3, 2, 1, 3, 0, 0, 0, 3,
        0, 3, 2])


# CNN